In [1]:
import time
import json
import sys
import os
import glob
import re
import logging
import traceback

from collections import deque

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client,
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer

# FuncX Service Client ID (hidden in recent FuncX version
CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [2]:
# Globus Online Endpoints
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' # mona4 #'hostel' #aps/workstation
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta'

# FuncX endpoint at ThetaGPU (ALCF) and Prisma (APS)
theta_fx_endpoint = '2ab22e1f-4cf1-47e9-a40c-dd8c58b41d73' #theta-ptycho-8w-1n #'512337eb-4d7d-47a7-b10e-2ae7cf94c793' # theta-ptycho-8w-1n-2cn
prisma_fx_endpoint = '4bf59543-3398-42d2-9416-c628e9f5635f' #mona4 (prisma-ptycho)

In [3]:
fxc = FuncXClient()

In [4]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    import logging
    from subprocess import PIPE

    rec_ngpu = data['rec_ngpu']
    wid = data['wid']
    dataset_name = data['dataset_name']

    log_file_name = "/grand/hp-ptycho/bicer/20210723_workflow-Xu/logs/funcx-ptycho-{}-w{}-g{}.log".format(dataset_name, wid, rec_ngpu)
    logging.basicConfig(filename=log_file_name,
                        filemode='a',
                        format='%(asctime)s %(levelname)s %(message)s',
                        level=logging.INFO,
                        datefmt='%Y-%m-%d %H:%M:%S')

    logging.info("Starting ptycho funcx function.")

    remote_log_file_name = "/grand/hp-ptycho/bicer/20210723_workflow-Xu/logs/tike-wf-{}-w{}-g{}.log".format(dataset_name, wid, rec_ngpu)

    python_path = data['python_path']
    script_path = data['script_path']

    #recon. script parameters
    ifpath = data['ifpath']
    pospath = data['position_path']
    probepath = data['probe_path']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    rec_overwrite = '--overwrite' if (('rec_overwrite' in data) and data['rec_overwrite']) else ''
    rec_auto_pin = '--auto-pin' if (('rec_auto_pin' in data) and data['rec_auto_pin']) else ''
    rec_gpu_id = data['rec_gpu_id']
    rec_log_filename = remote_log_file_name

    try:
        os.mkdir(ofpath)
    except:
        pass

    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' --position-path={pospath} --probe-path={probepath} {rec_overwrite} {rec_auto_pin} --gpu-id={rec_gpu_id} --folder='{ofpath}' --log-file='{rec_log_filename}'"
    logging.info(f"Running command: {cmd}")

    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr

func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

dcb1ddf2-3091-4d24-bc06-48c4571c861a


In [5]:
def get_folder_paths(path):
    import glob
    import re

    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join(reversed(v)).index('/'):]).group(0)))


def get_file_paths(path):
    import glob
    import re

    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join((reversed(v))).index('/'):v.index('.')]).group(0)))

fx_func_get_file_paths_uuid = fxc.register_function(get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

23c872a0-9acd-4592-a7fc-2c1405b9d3b2
d30e8b90-616b-4f9b-9fd9-d660e21b0f98


In [6]:
# Globus Automate flow definition

flow_definition = {
  "Comment": "Ptychographic reconstruction workflow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint",
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": True
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 14400,
      "Next": "PosTransfer"
    },
    "PosTransfer": {
      "Comment": "Transfer position file",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint",
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_pos_path",
            "destination_path.$": "$.input.dest_pos_path",
            "recursive": False
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 14400,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 14400,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint",
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 14400,
      "End": True
    },
  }
}

In [7]:
src_wf_root_path = '/mnt/micdata2/velociprobe/2021-2/Xu'
src_input_folder_prefix = "ptycho"
src_input_pos_folder_prefix = "positions"
src_output_folder_prefix = "wf-recons-i100"

dest_wf_root_path = '/grand/hp-ptycho/bicer/20210723_workflow-Xu'
dest_input_folder_prefix = "input"
dest_output_folder_prefix = "output"

add_larger_than = 1500

src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*"
rid = fxc.run(src_input_folder_paths_regex,
              endpoint_id=prisma_fx_endpoint,
              function_id=fx_func_get_folder_paths_uuid)
#src_input_folder_paths = fxc.get_result(rid)
while True:
    try:
        time.sleep(1)
        src_input_folder_paths = fxc.get_result(rid)
        #print(src_input_folder_paths)
    except KeyboardInterrupt:
        break
    except:
        print("Result is not ready:{}".format(sys.exc_info()[0]))
        continue
    break

iids = []
for src_input_folder_path in src_input_folder_paths:
    #print(src_input_folder_path)
    iid = re.findall(r'\d+', src_input_folder_path)
    if int(iid[-1]) > add_larger_than: 
        #print("id is being added:{}".format(int(iid[-1])))
        iids.append(iid[-1])
    else: continue
    

src_output_folder_paths_regex = f"{src_wf_root_path}/{src_output_folder_prefix}/*"
rid = fxc.run(src_output_folder_paths_regex,
              endpoint_id=prisma_fx_endpoint,
              function_id=fx_func_get_folder_paths_uuid)
while True:
    try:
        time.sleep(1)
        ex_src_output_folder_paths = fxc.get_result(rid)
        #print(ex_src_output_folder_paths)
    except KeyboardInterrupt:
        break
    except:
        print("Result is not ready:{}".format(sys.exc_info()[0]))
        continue
    break
    
oiids = []
for ex_src_output_folder_path in ex_src_output_folder_paths:
    oiid = re.findall(r'\d+', ex_src_output_folder_path)
    if int(oiid[-1]) > add_larger_than: 
        #print("id is being added:{}".format(int(oiid[-1])))
        oiids.append(oiid[-1])
    else: continue
    
diff_iids = []
for iid in iids:
    if (int(iid)>add_larger_than) and (iid not in oiids): 
        diff_iids.append(iid)

#print("Creating works for these ids: {}\n".format(diff_iids))


fsrc_input_folder_paths = []
src_input_pos_files = []
src_output_folder_paths = []
dest_output_folder_paths = []
dest_input_folder_paths = []
dest_input_pos_files = []
iids = []
for src_input_folder_path in src_input_folder_paths[:-1]:
    #print(src_input_folder_path)
    iid = re.findall(r'\d+', src_input_folder_path)
    if (int(iid[-1]) <= add_larger_than) or (iid[-1] in oiids): continue
    iids.append(iid[-1])
    fsrc_input_folder_paths.append(src_input_folder_path)
    src_input_pos_files.append(f"{src_wf_root_path}/{src_input_pos_folder_prefix}/fly{iid[-1]}_0.txt")
    src_output_folder_path = f"{src_wf_root_path}/{src_output_folder_prefix}/{iid[-1]}"
    src_output_folder_paths.append(src_output_folder_path)
    dest_input_folder_path = f"{dest_wf_root_path}/{dest_input_folder_prefix}/{iid[-1]}"
    dest_input_folder_paths.append(dest_input_folder_path)
    dest_input_pos_files.append(f"{dest_input_folder_path}/fly{iid[-1]}_0.txt")
    dest_output_folder_path = f"{dest_wf_root_path}/{dest_output_folder_prefix}/{iid[-1]}"
    dest_output_folder_paths.append(dest_output_folder_path)
src_input_folder_paths = fsrc_input_folder_paths
#print(len(src_input_folder_paths))
# src_input_folder_paths: diffraction patterh files to be processed @ APS
# src_output_folder_paths: folders for reconstrcuted images after processing @ APS
# dest_input_folder_paths: diffraction patterh files to be processed @ ALCF
# dest_output_folder_paths: folders for reconstrcuted images after processing @ ALCF

iids.reverse()
src_input_folder_paths.reverse()
src_input_pos_files.reverse()
src_output_folder_paths.reverse()
dest_input_folder_paths.reverse()
dest_input_pos_files.reverse()
dest_output_folder_paths.reverse()

for (iid, src_input_folder_path, src_input_pos_file, src_output_folder_path, dest_input_folder_path, dest_input_pos_file, dest_output_folder_path ) in zip(iids, src_input_folder_paths, src_input_pos_files, src_output_folder_paths, dest_input_folder_paths, dest_input_pos_files, dest_output_folder_paths):
    print(f"{iid}: Source input folder: {src_input_folder_path}; Source position file: {src_input_pos_file}; Source output folder: {src_output_folder_path}")
    print(f"Dest. input folder: {dest_input_folder_path}; Dest. position file: {dest_input_pos_file}; Dest. output folder: {dest_output_folder_path}")
    print()

2915: Source input folder: /mnt/micdata2/velociprobe/2021-2/Xu/ptycho/fly2915; Source position file: /mnt/micdata2/velociprobe/2021-2/Xu/positions/fly2915_0.txt; Source output folder: /mnt/micdata2/velociprobe/2021-2/Xu/wf-recons-i100/2915
Dest. input folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/2915; Dest. position file: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/2915/fly2915_0.txt; Dest. output folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/output/2915

2914: Source input folder: /mnt/micdata2/velociprobe/2021-2/Xu/ptycho/fly2914; Source position file: /mnt/micdata2/velociprobe/2021-2/Xu/positions/fly2914_0.txt; Source output folder: /mnt/micdata2/velociprobe/2021-2/Xu/wf-recons-i100/2914
Dest. input folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/2914; Dest. position file: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/2914/fly2914_0.txt; Dest. output folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/output/2914

2913: Source input folder: /mn

1861: Source input folder: /mnt/micdata2/velociprobe/2021-2/Xu/ptycho/fly1861; Source position file: /mnt/micdata2/velociprobe/2021-2/Xu/positions/fly1861_0.txt; Source output folder: /mnt/micdata2/velociprobe/2021-2/Xu/wf-recons-i100/1861
Dest. input folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/1861; Dest. position file: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/1861/fly1861_0.txt; Dest. output folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/output/1861

1860: Source input folder: /mnt/micdata2/velociprobe/2021-2/Xu/ptycho/fly1860; Source position file: /mnt/micdata2/velociprobe/2021-2/Xu/positions/fly1860_0.txt; Source output folder: /mnt/micdata2/velociprobe/2021-2/Xu/wf-recons-i100/1860
Dest. input folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/1860; Dest. position file: /grand/hp-ptycho/bicer/20210723_workflow-Xu/input/1860/fly1860_0.txt; Dest. output folder: /grand/hp-ptycho/bicer/20210723_workflow-Xu/output/1860

1859: Source input folder: /mn

In [9]:
# Ptycho recon params
script_path = '/home/bicer/projects/tike/scripts/tike-pinned-ptycho-wf.py'
python_path = "/home/bicer/projects/tyler/bin/python"

probe_path = '/grand/hp-ptycho/bicer/20210723_workflow-Xu/probes/velociprobe-probe.npy'

rec_alg = 'lstsq_grad'
rec_nmodes = 1
rec_upd_pos = False
rec_niter = 100
rec_output_freq = 20
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 1
rec_use_mpi = False
rec_overwrite = True
rec_auto_pin = True

dataset_name = "battery"

nworkers_per_node = 8

flow_inputs = []
gcounter = 0
for (iid, src_input_folder_path, src_input_pos_file, src_output_folder_path,
     dest_input_folder_path, dest_input_pos_file, dest_output_folder_path ) in zip(iids,
    src_input_folder_paths, src_input_pos_files, src_output_folder_paths,
    dest_input_folder_paths, dest_input_pos_files, dest_output_folder_paths):

    rec_gpu_id = gcounter%nworkers_per_node

    flow_input = {
        "input": {
            "source_endpoint": f"{src_endpoint}",
            "source_path": f"{src_input_folder_path}/",
            "source_pos_path": f"{src_input_pos_file}",
            "dest_endpoint": dest_endpoint,
            "dest_path": f"{dest_input_folder_path}/",
            "dest_pos_path": dest_input_pos_file,

            "result_path": f"{dest_output_folder_path}",
            "source_result_path": f"{src_output_folder_path}",
            "fx_ep": f"{theta_fx_endpoint}",
            "fx_id": f"{func_ptycho_uuid}",
            "params": {'ifpath': f"{dest_input_folder_path}/fly{iid}_master.h5",
                       'ofpath': f"{dest_output_folder_path}/",
                       'position_path': dest_input_pos_file,
                       'probe_path': probe_path,
                       'script_path': script_path,
                       'python_path': python_path,
                       'rec_alg': rec_alg,
                       'rec_nmodes': rec_nmodes,
                       'rec_upd_pos': rec_upd_pos,
                       'rec_niter': rec_niter,
                       'rec_output_freq': rec_output_freq,
                       'rec_recover_psi': rec_recover_psi,
                       'rec_recover_probe': rec_recover_probe,
                       'rec_recover_positions': rec_recover_positions,
                       'rec_model': rec_model,
                       'rec_ngpu': rec_ngpu,
                       'rec_use_mpi': rec_use_mpi,
                       'rec_overwrite': rec_overwrite,
                       'rec_auto_pin': rec_auto_pin,
                       'rec_gpu_id':rec_gpu_id,
                       'dataset_name': dataset_name,
                       'wid':gcounter}
        }
    }
    gcounter=gcounter+1
    flow_inputs.append(flow_input)

#print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")
#print(f"recon file:{dest_filepath}/{src_filename} output:{dest_resultpath}/")
#print(f"transfer file from {dest_endpoint}:{dest_resultpath} to {src_endpoint}#{src_result_path}/")

In [10]:
print(len(flow_inputs))

1304


In [11]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple ptycho data analysis flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

print(flow)

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/254d1fda-d6b0-44fc-b817-284287868c20', 'administered_by': [], 'api_version': '1.0', 'created_at': '2021-07-26T11:00:38.710127+00:00', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'definition': {'Comment': 'Ptychographic reconstruction workflow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input.params'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 14400}, 'PosTransfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Transfer position file', 'Next': 'An

In [12]:
log_file_name = "/home/beams/TBICER/logs/20210723_workflow-Xu/funcx-ptycho-wf-{}-g{}.log".format(dataset_name, rec_ngpu)
logging.basicConfig(filename=log_file_name,
                    filemode='a',
                    format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

q0 = deque()
q1 = deque()

logging.info(f"Workflow information for: {log_file_name}")
logging.info(f"Deployed flow: {flow}")
nnodes = 4
nworkers_per_node = 8

counter=0
nflows = len(flow_inputs)
logging.info(f"Starting workflow {dataset_name}:{nnodes} left={nflows-counter} succeeded={counter}")

for i in range(len(flow_inputs)):
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i])
        q1.append(flow_action)
        lstr=f"Flow {i} initiated and added to q1: {flow_action['action_id']}"
        logging.info(lstr)
        print(lstr)

i=-1
while len(q1)>0:
    i = (i+1)%len(q1)
    try:
        flow = flows_client.flow_action_status(flow_id, flow_scope, q1[i]['action_id'])
    except Exception as exc:
        err = traceback.format_exc()
        print(f"Exception:{exc}")
        logging.warning(err)
        continue
 
 
    lstr = f"len(q0)={len(q0)}; len(q1)={len(q1)}; i={i}"
    logging.info(lstr)
    print(lstr)
 
    lstr = f"Flow {i} status: {q1[i]['action_id']}: {flow['status']}"
    logging.info(lstr)
    print(lstr)
 
    if len(q1) < 4: time.sleep(1)
 
    if flow['status'] == 'SUCCEEDED':
        logging.info(f"Flow succeeded: {q1[i]}")
        del q1[i]
        counter = counter+1
        if len(q0)>0:
            flow_input = q0.popleft()
            flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
            q1.append(flow_action)
            lstr = f"New flow initiated and added to the q1: {flow_action['action_id']}"
            logging.info(lstr)
            print(lstr)
            lstr=f"Copy from {flow_input['input']['params']['ifpath']} to {flow_input['input']['params']['ofpath']}"
            logging.info(lstr)
            print(lstr)
 
    if flow['status'] == 'FAILED':
        lstr = f"Failed action, removing: {flow_action['action_id']}"
        print(lstr)
        logging.warning(lstr)
 
        del q1[i]

logging.info(f"Done workflow {dataset_name}:{nnodes} left={nflows-counter} succeeded={counter}")

Starting login with Globus Auth, press ^C to cancel.
Flow 0 initiated and added to q1: 07b53911-7473-4a3f-a3f9-adcf0f7066a6
Flow 1 initiated and added to q1: 50301192-a362-4d6b-9cb0-26da9956b5bd
Flow 2 initiated and added to q1: c9ccbcd5-c2f6-4071-96c3-3314a43aeada
Flow 3 initiated and added to q1: ee2fcdf3-c2a3-4458-bcf3-5b0ec9aee87e
Flow 4 initiated and added to q1: b8b3cfbf-a440-4904-8ea5-f6cf3f9e0cb0
Flow 5 initiated and added to q1: 8e6831c3-ad29-4121-9db9-7cbd9fe961ae
Flow 6 initiated and added to q1: 07963e94-5349-49d9-8bbc-77821c5fbbb7
Flow 7 initiated and added to q1: ae34cec1-e5a9-4392-8908-05e06be64e77
Flow 8 initiated and added to q1: 617a1bab-66cb-4c4b-88e1-e4a8ed6acd6c
Flow 9 initiated and added to q1: ac0d1071-ecdc-4651-9b30-30fea19f4cc5
Flow 10 initiated and added to q1: d7a8b4f5-ac9f-4462-a163-d138a9b1931f
Flow 11 initiated and added to q1: bc803d9a-c011-4cc1-87ab-a0c5d7dc0fbf
Flow 12 initiated and added to q1: 89ac787b-9300-4d6e-bbe6-4748794b862e
Flow 13 initiated and

Flow 114 initiated and added to q1: dae2610d-ad07-4b26-b280-88668017b7b6
Flow 115 initiated and added to q1: 1d67eb43-5905-4658-9cac-9586c40b4c5c
Flow 116 initiated and added to q1: eb33d205-a065-48ec-bfb9-2020465a7b72
Flow 117 initiated and added to q1: 7086be88-5bed-479a-abd4-a7f1bbcaef51
Flow 118 initiated and added to q1: d3d917bb-dbaf-4971-822a-8dfad032f4de
Flow 119 initiated and added to q1: 204520e4-9984-47ee-9ac0-44a65f279e7e
Flow 120 initiated and added to q1: 57a95626-9a73-4d75-b9ca-9a2164de1f5a
Flow 121 initiated and added to q1: f7fd5a62-35a7-4097-804c-657a5f59ea6e
Flow 122 initiated and added to q1: 9be1e9ae-0750-440e-b028-664051c0be40
Flow 123 initiated and added to q1: 06af481a-5946-46a9-9c91-5ffbb495bb82
Flow 124 initiated and added to q1: 45fa3932-76fe-4147-842c-3486b89bfce7
Flow 125 initiated and added to q1: a0626707-6361-4fba-873c-4736dba3de3a
Flow 126 initiated and added to q1: 990e86dd-e1b9-424c-a6c0-366dbe7a9489
Flow 127 initiated and added to q1: a6e8707c-7afb-4

Flow 227 initiated and added to q1: 582b84f8-05e7-46ed-87de-58d8026d52a0
Flow 228 initiated and added to q1: 841dc30a-e8a5-4a71-a62b-10300be4a8d7
Flow 229 initiated and added to q1: 1afbc496-0033-4771-9fc6-8cd02955e38d
Flow 230 initiated and added to q1: 9e90b914-6e8b-420e-acaa-4b6d8f02a008
Flow 231 initiated and added to q1: c02564bd-06a5-46a0-896a-8842c300f2ce
Flow 232 initiated and added to q1: 472ae57e-2556-4d5f-aef9-e4983cb20f19
Flow 233 initiated and added to q1: 19938b4f-d7a4-4807-8744-65a6b3952108
Flow 234 initiated and added to q1: 5b4c1f2b-0c6a-4963-9f56-6e08e6d565cc
Flow 235 initiated and added to q1: e0b47ad5-b706-4873-8c7d-0139c2501a78
Flow 236 initiated and added to q1: 8b3b230a-ed65-48fe-99c0-4b3bc9fe43ed
Flow 237 initiated and added to q1: ad6bf617-d0be-40a6-9904-3c356efc1116
Flow 238 initiated and added to q1: d0a4d9b2-d4d4-4d78-a153-859b1f7a0997
Flow 239 initiated and added to q1: aa177240-a45e-4b31-9567-2b5901e5035f
Flow 240 initiated and added to q1: e30bef4a-6940-4

Flow 340 initiated and added to q1: 2bfc9736-068d-4373-bd7f-83425a1f715d
Flow 341 initiated and added to q1: 63812099-8a81-4fe8-986c-f53b94f3afe7
Flow 342 initiated and added to q1: 80380209-dd50-4cda-af03-5f4576b8493b
Flow 343 initiated and added to q1: 43cf2a7f-b2ad-4327-b44e-6e8e1712a5fc
Flow 344 initiated and added to q1: 91162d86-23f2-48b8-8ed0-d5a01a93dfd1
Flow 345 initiated and added to q1: b4133d4c-d12a-49b1-906f-97deafe6af5a
Flow 346 initiated and added to q1: 721b6770-7b25-4a0c-a8ae-84830bc28afd
Flow 347 initiated and added to q1: fca4abca-88c5-4676-bdb3-0df399f372eb
Flow 348 initiated and added to q1: 70c42cc2-47b7-4307-a702-7ce3981fab86
Flow 349 initiated and added to q1: f499f4a6-1910-4789-959e-00f22c0f42dd
Flow 350 initiated and added to q1: 386290e6-fcca-4316-9c87-e9c51218a5ca
Flow 351 initiated and added to q1: 64023d7b-dfa0-49ee-8257-c61501968d6e
Flow 352 initiated and added to q1: c9ed19bf-da0d-4a7d-bc63-99072841749d
Flow 353 initiated and added to q1: dbec81f7-126d-4

Flow 453 initiated and added to q1: b3a2014e-9986-44ca-a37b-056263531050
Flow 454 initiated and added to q1: 5c86ecbb-f0f2-469c-8632-ebab3b1cf813
Flow 455 initiated and added to q1: 1005762c-1be8-4059-9d37-34ce8bd35137
Flow 456 initiated and added to q1: 57f02c03-23d1-4661-a3eb-968c6fad38ab
Flow 457 initiated and added to q1: 67ed1ea7-555b-4e97-9191-1c2e07c4e6ba
Flow 458 initiated and added to q1: 07aac793-214e-4596-9cf3-4a7f56476a65
Flow 459 initiated and added to q1: 5c918369-34e8-4262-bc6b-ce1ed74ec8b8
Flow 460 initiated and added to q1: 0dee92e7-bfe4-477e-b291-a618c7c8dff4
Flow 461 initiated and added to q1: 8c037b13-83de-42e4-a116-0c5492da3148
Flow 462 initiated and added to q1: c0678d30-058d-437d-aaf2-09f4c4ba04ab
Flow 463 initiated and added to q1: a75ab5c8-8e28-4e05-b695-1877a96509a2
Flow 464 initiated and added to q1: a56ac242-4a53-489b-9b43-385753a2b963
Flow 465 initiated and added to q1: aec3f534-8c85-4e9e-97df-c74d4ca16d1d
Flow 466 initiated and added to q1: 3cd50125-586d-4

Flow 566 initiated and added to q1: 370945c6-d657-4809-9a5c-f5bb11eb9c04
Flow 567 initiated and added to q1: fb328a24-248d-46c6-9f60-b437906cecc4
Flow 568 initiated and added to q1: de6d248a-fe57-4e08-a056-8b32a938c9a5
Flow 569 initiated and added to q1: e925f4ef-e639-43cb-bd6b-eaef0157ff9d
Flow 570 initiated and added to q1: 434dba94-0570-4fda-bf7a-4977f73efb81
Flow 571 initiated and added to q1: 7beed91e-814c-4402-bd94-ea9456da4e1b
Flow 572 initiated and added to q1: ff9d98f1-ac66-4a03-833d-c48ca3c40a24
Flow 573 initiated and added to q1: 684f7fa3-3ed8-4810-a3cb-99b56dbc4d73
Flow 574 initiated and added to q1: 3a7320e6-665d-4edb-b4b2-22706fd3c1bb
Flow 575 initiated and added to q1: 80b9bd6a-9c59-4645-b821-d74f18b638bc
Flow 576 initiated and added to q1: b8c60481-f051-462b-a28e-f4dd60e7de48
Flow 577 initiated and added to q1: bd89e14d-77c8-451e-ba40-61141bae6eb1
Flow 578 initiated and added to q1: cde6f6b1-e132-4450-a868-b183268db792
Flow 579 initiated and added to q1: 54c2369b-ba51-4

Flow 679 initiated and added to q1: 4cebee43-f33c-40b8-91da-d09621b46bfe
Flow 680 initiated and added to q1: 0c263330-b520-4719-9066-a74d5820c3d3
Flow 681 initiated and added to q1: e8cf0143-2b82-4bef-b7b0-e128c1205c97
Flow 682 initiated and added to q1: a9eacd57-13dc-4f29-8a94-4cc5cea159a3
Flow 683 initiated and added to q1: eb363fc1-a81d-407e-9a00-da0c70a4c4f8
Flow 684 initiated and added to q1: b79e18e2-3576-4aad-9373-388e12562a15
Flow 685 initiated and added to q1: 3e0f2b71-c34c-4792-bd8e-e9226e2768ff
Flow 686 initiated and added to q1: 3fd678bf-beca-4196-8a5c-49b8e19b272f
Flow 687 initiated and added to q1: 458f8d29-f57f-442b-9e0e-d908b9a3d471
Flow 688 initiated and added to q1: bf83bb3c-34cb-45dd-a1b8-ef797cb672c8
Flow 689 initiated and added to q1: aef587ed-216b-4c0d-8ca0-4728cf49c7b3
Flow 690 initiated and added to q1: 8cc3fc85-88bb-4727-a809-c41952ab30db
Flow 691 initiated and added to q1: 23995756-f457-467f-950a-cbd2b7315473
Flow 692 initiated and added to q1: cb751fc5-f49b-4

Flow 792 initiated and added to q1: 5e29f7b9-2943-452e-8b4f-80a77294dcdb
Flow 793 initiated and added to q1: c87105b4-dcaf-4d2e-80a2-c5c5ea65126f
Flow 794 initiated and added to q1: 7d14c0bf-5b8d-4519-9b9d-460134346246
Flow 795 initiated and added to q1: fcb41812-1501-41f7-b23f-15411abc734a
Flow 796 initiated and added to q1: a7c8368f-56c8-4d07-9113-112eda039f9a
Flow 797 initiated and added to q1: f0afaf6d-8e16-4edc-bc5c-3c89165d2dbe
Flow 798 initiated and added to q1: f8fe894b-4f19-4dfb-b738-ea05dc9170fb
Flow 799 initiated and added to q1: ebc3ee92-213c-475b-94ee-43c755292e49
Flow 800 initiated and added to q1: 9e8b2d21-ae93-41a7-a42d-ef25a9269445
Flow 801 initiated and added to q1: 5a6fe583-e0c2-4b94-b70d-9f1ab7eb728c
Flow 802 initiated and added to q1: f97046a2-000d-4c45-9919-b75bfbf3a610
Flow 803 initiated and added to q1: 88472e19-2dfb-4ca3-8227-f622d6ba3cb1
Flow 804 initiated and added to q1: a233935a-cfc7-4bac-b6cd-e0db31f2b513
Flow 805 initiated and added to q1: 71e6cda7-39d5-4

Flow 905 initiated and added to q1: 63123d08-d951-422e-a8d6-3d504fa11b18
Flow 906 initiated and added to q1: a3877840-ebf1-409e-a3fb-5011df0ff95d
Flow 907 initiated and added to q1: 57b4aeef-3f83-4540-9de7-3a6c41ad7d58
Flow 908 initiated and added to q1: 9d3e1211-a50b-46b7-be3b-dc2e5a91ad79
Flow 909 initiated and added to q1: bfcbc029-3ddf-40c6-8396-676f267e4af4
Flow 910 initiated and added to q1: c1024a8d-71f2-4e9f-a150-905692083210
Flow 911 initiated and added to q1: 0113adcd-b24d-4830-b566-8abcaeea65d6
Flow 912 initiated and added to q1: ae8c9446-d1cd-4dbc-a819-cb5381bfb81b
Flow 913 initiated and added to q1: 1520f9a1-5cf7-43f6-b959-dfcd6a0fdaf0
Flow 914 initiated and added to q1: 74e02ab8-16d3-4081-b106-411b0b819dbe
Flow 915 initiated and added to q1: ab3f2ff1-ff8e-4dbf-8e91-3823a54a7997
Flow 916 initiated and added to q1: 166a807e-567c-4ea7-9ce8-23a831424ceb
Flow 917 initiated and added to q1: 7da36ee1-1eb8-43f1-9b28-36b8f8b27393
Flow 918 initiated and added to q1: 9985440c-39e7-4

GlobusConnectionError: ConnectionError on request

In [ ]:
print(flow_scope)

In [ ]:
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])